In [21]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Label dictionary
labels_diction = {
    'Bacterial leaf blight': 0,
    'Brown spot': 1,
    'Leaf smut': 2,
}

data_dir = '/content/drive/MyDrive/DataSets/rice_leaf_diseases'

X = []
y = []
for folder in os.listdir(data_dir):
    label = labels_diction[folder]
    for image in os.listdir(os.path.join(data_dir, folder)):
        image_path = os.path.join(data_dir, folder, image)
        img = cv2.imread(image_path)
        img = cv2.resize(img, (224, 224))
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)


In [23]:
# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = datagen.flow(X, y, batch_size=32)

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(labels_diction), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])


In [ ]:
# Model training
history = model.fit(train_generator, epochs=20)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow(X, y, batch_size=32)
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test loss:', test_loss)
print('Test accuracy:', test_accuracy)


Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.3937 - loss: 1.3034
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 102ms/step - accuracy: 0.3232 - loss: 1.3079
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.3216 - loss: 1.3405
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.3159 - loss: 1.2887
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.4296 - loss: 1.2033
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - accuracy: 0.3463 - loss: 1.1753
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.3413 - loss: 1.2569
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.2738 - loss: 1.2665
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.2840 - loss: 1.2765
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - accuracy: 0.2706 - loss: 1.2807
Epoch 11/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.3584 - loss: 1.1823
Epoch 12/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - accuracy: 0.3552 - loss: 1.1968
E

In [ ]:
# Plot training accuracy
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the model
model.save('rice_leaf_diseases_model_resnet50.h5')


In [ ]:
from PIL import Image  # Import PIL for image handling

new_model = tf.keras.models.load_model('rice_leaf_diseases_model_resnet50.h5')

image_path = '/content/drive/MyDrive/DataSets/rice_leaf_diseases/Brown spot/DSC_0100.jpg'
test_image = Image.open(image_path)
resized_image = test_image.resize((224, 224))
resized_image = np.array(resized_image) / 255.0
resized_image = resized_image[np.newaxis, ...]
prediction = new_model.predict(resized_image)
predicted_class_index = np.argmax(prediction)
print('Predicted class index:', predicted_class_index)

# Map predicted class index to the actual label
predicted_label = {v: k for k, v in labels_diction.items()}.get(predicted_class_index)
print('Predicted label:', predicted_label)
